In [ ]:
import numpy as np
import os 
import pandas as pd
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.models import Model, Sequential,load_model
from tensorflow.keras.layers import Conv2D, Input, Dense, Flatten, MaxPooling2D, Dropout, AveragePooling2D, BatchNormalization
from tensorflow.keras.callbacks import ModelCheckpoint

In [ ]:
train_data=np.load('...\\HAR\\Preprocessed Data\\Train_Data.npz')
train_data.files

In [ ]:
x_train=train_data['x_train']
y_train=train_data['y_train']
print(x_train.shape)
print(y_train.shape)

In [ ]:
test_data=np.load('...\\HAR\\Preprocessed Data\\Test_Data.npz')
test_data.files

In [ ]:
x_test=test_data['x_test']
y_test=test_data['y_test']
print(x_test.shape)
print(y_test.shape)

# Spatial Attention Module (SAM)

In [ ]:
class Spatial_Attention_Block(layers.Layer):
    def __init__(self,out_channel,kernel,**kwargs):
        super(Spatial_Attention_Block,self).__init__(**kwargs)
        
        self.out_channel=out_channel
        self.kernel=kernel
        #1,16,32     1,3,3
        self.conv_1=Conv2D(filters=self.out_channel[0],kernel_size=self.kernel[0])
        self.conv_2=Conv2D(filters=self.out_channel[1],kernel_size=self.kernel[1],padding='same')
        self.conv_3=Conv2D(filters=self.out_channel[2],kernel_size=self.kernel[2],padding='same')
        
    def call(self,images):
        x=tf.nn.relu(self.conv_1(images))
        #print("For 1st Attention conv-1 : ",x.shape)
        x=tf.nn.relu(self.conv_2(x))
        #print("For 1st Attention conv-2 : ",x.shape)
        x=tf.nn.relu(self.conv_3(x))
        #print("For 1st Attention conv-3 : ",x.shape)
        
        return (x+images)
    
    def get_config(self):
        config = super(Spatial_Attention_Block,self).get_config()
        config.update({
            "out_channel": self.out_channel,
            "kernel": self.kernel,
             })
        return config
    
    @classmethod
    def from_config(cls, config):
        return cls(**config)
    

# Model

In [ ]:
model=Sequential([
    
            Conv2D(filters=32,kernel_size=(3,3),input_shape=(128,128,6),activation='relu'),
            Spatial_Attention_Block(out_channel=[1,16,32],kernel=[1,3,3]),
            MaxPooling2D((2,2)),
            Dropout(0.2),
    
            Conv2D(filters=64,kernel_size=(3,3),activation='relu'),
            Spatial_Attention_Block(out_channel=[1,32,64],kernel=[1,3,3]),
            MaxPooling2D((2,2)),
            Dropout(0.2),
    
            Conv2D(filters=64,kernel_size=(3,3),activation='relu'),
            Spatial_Attention_Block(out_channel=[1,32,64],kernel=[1,3,3]),
            MaxPooling2D((2,2)),
            Dropout(0.2),
    
            Conv2D(filters=128,kernel_size=(3,3),activation='relu'),
            Spatial_Attention_Block(out_channel=[1,64,128],kernel=[1,3,3]),
            MaxPooling2D((2,2)),
            Dropout(0.2),
    
            Flatten(),
            Dense(1024,activation='relu'),
            Dropout(0.2),
            Dense(6,activation='softmax')
])

model.summary()

In [ ]:
model.compile(loss=tf.keras.losses.sparse_categorical_crossentropy,
              optimizer=tf.keras.optimizers.Adam(),
              metrics=['accuracy'])

In [ ]:
filepath = '...\\HAR\\Model\\model-ep{epoch:03d}-val_loss{val_loss:.3f}-val_acc{val_accuracy:.3f}.h5'
checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=2, save_best_only=True, mode='min')

In [ ]:
history=model.fit( x_train,y_train,
                   batch_size=32,
                   epochs=150,
                   shuffle=True,
                  callbacks=[checkpoint],
                   validation_data=(x_test,y_test) )

# Learning Graph

In [ ]:
def plot_learning_curve(history,epochs):
    epoch_range=range(1,epochs+1)
    plt.plot(epoch_range,history.history['accuracy'])
    plt.plot(epoch_range,history.history['val_accuracy'])
    plt.title('Model Accuracy')
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy')
    plt.legend(['Train','Test'],loc='lower right')
    plt.show()
    
    plt.plot(epoch_range,history.history['loss'])
    plt.plot(epoch_range,history.history['val_loss'])
    plt.title('Model Loss')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend(['Train','Test'],loc='upper right')
    plt.show()
    
plot_learning_curve(history,150)    

# Extracting Features

In [ ]:
model=load_model('...\\HAR\\Model\\model-ep086-val_loss0.064-val_acc0.990.h5',custom_objects={'Spatial_Attention_Block':Spatial_Attention_Block})

In [ ]:
model.layers

In [ ]:
dense_layer=model.layers[17]
dense_layer

In [ ]:
from tensorflow.keras import backend

In [ ]:
feature_extractor=backend.function([model.input],[dense_layer.output])

In [ ]:
train_features=feature_extractor(x_train)
train_features=np.array(train_features[0])

In [ ]:
test_features=feature_extractor(x_test)
test_features=np.array(test_features[0])

# Classification Using KNN

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report
classes_= ['bike', 'sit', 'stairsdown', 'stairsup', 'stand', 'walk']
clf=KNeighborsClassifier(n_neighbors=5)
clf.fit(train_features,y_train)
y_pred = clf.predict(test_features)
report=classification_report(y_test,y_pred,target_names=classes_)
print(report)